In [9]:
%pip install pandas numpy scikit-learn pyarrow fastparquet

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pickle

In [11]:
# Carregar os landmarks e rótulos do arquivo .parquet
landmarks_df = pd.read_parquet('/home/viniciuss/Documents/projetos/FER-2013_emotion_detection/landmarks_balanced.parquet')

# Exibir as primeiras linhas do DataFrame para verificar
print("Primeiras linhas do DataFrame:")
print(landmarks_df.head())

Primeiras linhas do DataFrame:
          0         1         2         3         4         5         6  \
0  0.532348  0.831424 -0.026991  0.536164  0.778881 -0.125715  0.534156   
1  0.444280  0.714994 -0.065770  0.453648  0.612833 -0.181052  0.453395   
2  0.642857  0.702614 -0.061711  0.650431  0.612668 -0.130811  0.639840   
3  0.449421  0.796463 -0.033062  0.441775  0.698263 -0.173027  0.452383   
4  0.433063  0.768405 -0.041844  0.461766  0.688208 -0.187343  0.454095   

          7         8         9  ...      1395      1396      1397      1398  \
0  0.786114 -0.047851  0.510611  ...  0.585814  0.567389 -0.059839  0.753049   
1  0.636534 -0.080809  0.435849  ...  0.539301  0.412650 -0.059697  0.743716   
2  0.636721 -0.064377  0.616066  ...  0.654582  0.460700 -0.016876  0.772203   
3  0.711487 -0.063360  0.424534  ...  0.537971  0.429226 -0.095272  0.764291   
4  0.698178 -0.072511  0.452261  ...  0.571927  0.433360 -0.085450  0.795742   

       1399      1400      1401      

In [12]:
# Separar features e targets
X = landmarks_df.drop('label', axis=1).values
y = landmarks_df['label'].values

# Dividir dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizar as features se necessário (dependendo do modelo utilizado)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verificar tamanhos dos conjuntos
print(f"Tamanho do conjunto de treino: {len(X_train)} amostras")
print(f"Tamanho do conjunto de teste: {len(X_test)} amostras")

Tamanho do conjunto de treino: 12265 amostras
Tamanho do conjunto de teste: 3067 amostras


# Treinamentos SVC, LDA e RFC

In [13]:
# Inicializar o classificador SVC
svc = SVC(random_state=42)

# Treinar o modelo
svc.fit(X_train_scaled, y_train)

# Prever no conjunto de teste
y_pred_svc = svc.predict(X_test_scaled)

# Avaliar o desempenho
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print(f"Acurácia do SVC: {accuracy_svc:.2f}")

# Relatório de classificação
print("\nRelatório de Classificação do SVC:")
print(classification_report(y_test, y_pred_svc))

Acurácia do SVC: 0.65

Relatório de Classificação do SVC:
              precision    recall  f1-score   support

           0       0.58      0.55      0.56       767
           1       0.53      0.58      0.55       766
           2       0.73      0.74      0.74       767
           3       0.78      0.74      0.76       767

    accuracy                           0.65      3067
   macro avg       0.66      0.65      0.65      3067
weighted avg       0.66      0.65      0.65      3067



In [14]:
# Inicializar o LDA
lda = LinearDiscriminantAnalysis()

# Treinar o modelo
lda.fit(X_train_scaled, y_train)

# Prever no conjunto de teste
y_pred_lda = lda.predict(X_test_scaled)

# Avaliar o desempenho
accuracy_lda = accuracy_score(y_test, y_pred_lda)
print(f"Acurácia do LDA: {accuracy_lda:.2f}")

# Relatório de classificação
print("\nRelatório de Classificação do LDA:")
print(classification_report(y_test, y_pred_lda))

Acurácia do LDA: 0.69

Relatório de Classificação do LDA:
              precision    recall  f1-score   support

           0       0.58      0.61      0.60       767
           1       0.59      0.60      0.60       766
           2       0.80      0.80      0.80       767
           3       0.82      0.76      0.79       767

    accuracy                           0.69      3067
   macro avg       0.70      0.69      0.69      3067
weighted avg       0.70      0.69      0.69      3067



In [15]:
# Inicializar o RFC
rfc = RandomForestClassifier(random_state=42)

# Treinar o modelo
rfc.fit(X_train_scaled, y_train)

# Prever no conjunto de teste
y_pred_rfc = rfc.predict(X_test_scaled)

# Avaliar o desempenho
accuracy_rfc = accuracy_score(y_test, y_pred_rfc)
print(f"Acurácia do RFC: {accuracy_rfc:.2f}")

# Relatório de classificação
print("\nRelatório de Classificação do RFC:")
print(classification_report(y_test, y_pred_rfc))

Acurácia do RFC: 0.61

Relatório de Classificação do RFC:
              precision    recall  f1-score   support

           0       0.50      0.48      0.49       767
           1       0.52      0.54      0.53       766
           2       0.75      0.75      0.75       767
           3       0.69      0.69      0.69       767

    accuracy                           0.61      3067
   macro avg       0.61      0.61      0.61      3067
weighted avg       0.61      0.61      0.61      3067



# Salvando

In [16]:
model_path = '/home/viniciuss/Documents/projetos/FER-2013_emotion_detection/lda_model.pkl'

# Salvar o modelo LDA treinado
with open(model_path, 'wb') as file:
    pickle.dump(lda, file)

print(f"Modelo LDA salvo em {model_path}")

Modelo LDA salvo em /home/viniciuss/Documents/projetos/FER-2013_emotion_detection/lda_model.pkl
